In [1]:
from pathlib import Path
import tiffile as tif
import numpy as np
import multiscale.utility_functions as util
import os

In [2]:
def save_us_image(path, bmode, spacing):
    print('Saving {}'.format(path))
    ijstyle = bmode
    shape = ijstyle.shape
    ijstyle.shape = 1, shape[0], 1, shape[1], shape[2], 1

    tif.imwrite(path, ijstyle.astype(np.float32), imagej=True,
                resolution=(1./spacing[0], 1./spacing[1]),
                metadata={'spacing': spacing[2], 'unit': 'um'})
    
def open_image(path):
    return util.load_mat(str(path))
    
def get_spacing(data_dict):
    shape = np.shape(data_dict['IQData'])
    params = data_dict['param']
    lat_spacing = params['BmodeWidth']/shape[1]
    ax_spacing = params['BmodeDepth']/shape[0]
    spacing = [lat_spacing*1000, ax_spacing*1000, params['StepSize3D']*1000]
    return spacing

def iq_to_bmode(IQData):
    bmode_rotated = 20*np.log10(np.abs(IQData)+1)
    bmode = np.transpose(bmode_rotated, [2, 0, 1])
    return bmode

def convert_to_tif(path, output_path):
    data = open_image(path)
    bmode = iq_to_bmode(data['IQData'])
    spacing = get_spacing(data)
    save_us_image(output_path, bmode, spacing)
    
    

In [3]:
output_folder = r'C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files\Images'

In [16]:
name_list = ['Mammary over 1% agarose', 'Mammary over sand', 'ROI 1', 'ROI 2', 'ROI 3', 'ROI 3 - water', 'ROI 2 - water', 
             'ROI 1 - water']

base_path = r'C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files'

In [17]:
transducer_names = ['MS700', 'MS400', 'MS250', 'MS550D']


In [18]:
for name in range(len(transducer_names)):
    acquisition_folder = Path(output_folder, transducer_names[name])
    os.makedirs(str(acquisition_folder), exist_ok=True)
    for image in range(len(name_list)):
        input_path = Path(base_path, transducer_names[name], name_list[image] + '.mat')
        output_path = Path(output_folder, transducer_names[name], name_list[image] + '.tif')
        if output_path.is_file():
            continue
        convert_to_tif(input_path, output_path)

Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files\Images\Glass Bead Phantom\MS700.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files\Images\Glass Bead Phantom\MS400.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files\Images\Glass Bead Phantom\MS250.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files\Images\Glass Bead Phantom\MS550D.tif


In [ ]:
np.shape(data_dict['IQData'])